In [1]:
import numpy as np
import heapq

In [2]:

class Graph:
  def __init__(self, numVertices):
    self.numVertices = numVertices
    self.adjMatrix = np.full((numVertices, numVertices), np.inf)
    self.minPaths = np.full((numVertices, 2), np.inf)

  def addEdge(self, src, dest, weight):
    self.adjMatrix[src][dest] = weight
    self.adjMatrix[dest][src] = weight

  def getWeight(self, src, dest):
    return self.adjMatrix[src][dest]
  
  def getNumVertices(self):
    return self.numVertices
  
  def printGraph(self):
    print(self.adjMatrix)

  def getNeighbors(self, vertex):
    indices = np.where(self.adjMatrix[vertex] != np.inf)
    neighbors = [(i, self.adjMatrix[vertex][i]) for i in indices[0]]
    return neighbors
  
  def computeMinPaths(self):
    for i in range(self.numVertices):
      sortedIndices = np.argsort(self.adjMatrix[i])
      self.minPaths[i] = (self.adjMatrix[i][sortedIndices[0]], self.adjMatrix[i][sortedIndices[1]])

  def getOnePath(self):
    path = np.array([0])
    cost = 0
    for i in range(1, self.numVertices):
      cost += self.adjMatrix[path[-1]][i]
      path = np.concatenate([path, [i]]).astype(int)
    cost += self.adjMatrix[path[-1]][0]
    path = np.concatenate([path, [0]]).astype(int)
    return path, cost



class Node:
  def __init__(self, graph, prevNode=None, finalVertice=None):
    if prevNode is None or finalVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [finalVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph.getWeight(prevNode.path[-1], finalVertice)
    self.bound = bound(graph, finalVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"


def bound(graph, finalVertice, prevNode=None):
  if prevNode is None:
    return np.sum(graph.minPaths) / 2
  
  cost = graph.getWeight(prevNode.path[-1], finalVertice)

  bound = prevNode.bound * 2

  if cost >= graph.minPaths[finalVertice][1]:
    bound -= graph.minPaths[finalVertice][1]
    bound += cost
  
  if cost >= graph.minPaths[prevNode.path[-1]][1]:
    bound -= graph.minPaths[prevNode.path[-1]][1]
    bound += cost

  return bound / 2


def branchBoundTravelingSalesman(graph):
  graph.computeMinPaths()

  root = Node(graph, None, 0)
  queue = [root]
  heapq.heapify(queue)

  solution, best = graph.getOnePath()

  numVertices = graph.getNumVertices()

  while queue:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      if node.level <= numVertices:
        unvisited = np.setdiff1d(np.arange(numVertices), node.path)
        for k in unvisited:
          newNode = Node(graph, node, k)
          if newNode.bound < best and graph.getWeight(node.path[-1], k) != np.inf:
            heapq.heappush(queue, Node(graph, node, k))

  return solution, best

In [3]:
g = Graph(5)
g.addEdge(0, 1, 3)
g.addEdge(0, 2, 1)
g.addEdge(0, 3, 5)
g.addEdge(0, 4, 8)
g.addEdge(1, 2, 6)
g.addEdge(1, 3, 7)
g.addEdge(1, 4, 9)
g.addEdge(2, 3, 4)
g.addEdge(2, 4, 2)
g.addEdge(3, 4, 3)

# g.printGraph()
# print(g.getNeighbors(0))
# print(g.getNeighbors(1))
# print(g.getNeighbors(2))
# print(g.getNeighbors(3))
# print(g.getNeighbors(4))

g.computeMinPaths()
print(g.minPaths)

print('solution', branchBoundTravelingSalesman(g))


[[1. 3.]
 [3. 6.]
 [1. 2.]
 [3. 4.]
 [2. 3.]]
solution (array([0, 2, 4, 3, 1, 0]), 16.0)


In [4]:
g1 = Graph(4)
g1.adjMatrix = np.array([[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]])

g1.printGraph()
g1.computeMinPaths()
print(g1.minPaths)

print('solution', branchBoundTravelingSalesman(g1))

[[ 0 10 15 20]
 [10  0 35 25]
 [15 35  0 30]
 [20 25 30  0]]
[[ 0. 10.]
 [ 0. 10.]
 [ 0. 15.]
 [ 0. 20.]]
solution (array([0, 1, 3, 2, 0]), 80)


In [5]:
class Point:
  def __init__(self, id, x, y):
    self.id = id
    self.x = x
    self.y = y

  def distance(self, point):
    return np.sqrt((self.x - point.x)**2 + (self.y - point.y)**2)
  
  def __str__(self) -> str:
    return f'Point {self.id} ({self.x}, {self.y})'

In [6]:
# function to read a file and return a list of lists
def readFile(file_name):
    with open(file_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content if x[0].isdigit()]
    return content


In [7]:
# each element of the content list is a string with the id, x and y coordinates
# we need to split each line and create a Point object

content = readFile('berlin52.tsp')

points = []
for line in content:
  id, x, y = line.split(' ')
  points.append(Point(int(id)-1, float(x), float(y)))

for p in points:
  print(p)

Point 0 (565.0, 575.0)
Point 1 (25.0, 185.0)
Point 2 (345.0, 750.0)
Point 3 (945.0, 685.0)
Point 4 (845.0, 655.0)
Point 5 (880.0, 660.0)
Point 6 (25.0, 230.0)
Point 7 (525.0, 1000.0)
Point 8 (580.0, 1175.0)
Point 9 (650.0, 1130.0)
Point 10 (1605.0, 620.0)
Point 11 (1220.0, 580.0)


In [8]:
graph = Graph(len(points))

for p in points:
  for q in points:
    if p.id != q.id:
      graph.addEdge(p.id, q.id, p.distance(q))


In [9]:
graph.computeMinPaths()
print(graph.minPaths)

for line in graph.adjMatrix:
  print(line)

[[281.11385594 291.20439557]
 [ 45.         649.32657423]
 [281.11385594 308.05843601]
 [ 69.64194139 104.40306509]
 [ 35.35533906 104.40306509]
 [ 35.35533906  69.64194139]
 [ 45.         610.5735009 ]
 [180.34688797 183.43936328]
 [ 83.21658489 183.43936328]
 [ 83.21658489 180.34688797]
 [387.07234466 663.19303374]
 [294.36372059 349.28498393]]
[          inf  666.10809934  281.11385594  395.6008089   291.20439557
  326.26676202  640.8002809   426.87820277  600.18747071  561.47128155
 1040.97310244  655.01908369]
[ 666.10809934           inf  649.32657423 1047.09120902  945.14549145
  978.08486339   45.          956.15113868 1134.95594628 1132.98278892
 1638.78766166 1258.59048145]
[ 281.11385594  649.32657423           inf  603.51056329  508.9449872
  542.51728083  610.5735009   308.05843601  485.64390246  487.26276279
 1266.6885963   891.36131843]
[ 395.6008089  1047.09120902  603.51056329           inf  104.40306509
   69.64194139 1026.36494484  525.          611.00327331  533.900

In [10]:
print('solution', branchBoundTravelingSalesman(graph))

solution (array([ 0,  1,  6,  2,  7,  8,  9, 10, 11,  3,  5,  4,  0]), 4056.6809443061275)


In [11]:
# Python3 program to solve 
# Traveling Salesman Problem using 
# Branch and Bound.
import math
maxsize = float('inf')

# Function to copy temporary solution
# to the final solution
def copyToFinal(curr_path):
	final_path[:N + 1] = curr_path[:]
	final_path[N] = curr_path[0]

# Function to find the minimum edge cost 
# having an end at the vertex i
def firstMin(adj, i):
	min = maxsize
	for k in range(N):
		if adj[i][k] < min and i != k:
			min = adj[i][k]

	return min

# function to find the second minimum edge 
# cost having an end at the vertex i
def secondMin(adj, i):
	first, second = maxsize, maxsize
	for j in range(N):
		if i == j:
			continue
		if adj[i][j] <= first:
			second = first
			first = adj[i][j]

		elif(adj[i][j] <= second and
			adj[i][j] != first):
			second = adj[i][j]

	return second

# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving
# in the search space tree
# curr_path[] -> where the solution is being stored
# which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight, 
			level, curr_path, visited):
	global final_res
	
	# base case is when we have reached level N 
	# which means we have covered all the nodes once
	if level == N:
		
		# check if there is an edge from
		# last vertex in path back to the first vertex
		if adj[curr_path[level - 1]][curr_path[0]] != 0:
			
			# curr_res has the total weight
			# of the solution we got
			curr_res = curr_weight + adj[curr_path[level - 1]]\
										[curr_path[0]]
			if curr_res < final_res:
				copyToFinal(curr_path)
				final_res = curr_res
		return

	# for any other level iterate for all vertices
	# to build the search space tree recursively
	for i in range(N):
		
		# Consider next vertex if it is not same 
		# (diagonal entry in adjacency matrix and 
		# not visited already)
		if (adj[curr_path[level-1]][i] != 0 and
							visited[i] == False):
			temp = curr_bound
			curr_weight += adj[curr_path[level - 1]][i]

			# different computation of curr_bound 
			# for level 2 from the other levels
			if level == 1:
				curr_bound -= ((firstMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)
			else:
				curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)

			# curr_bound + curr_weight is the actual lower bound 
			# for the node that we have arrived on.
			# If current lower bound < final_res, 
			# we need to explore the node further
			if curr_bound + curr_weight < final_res:
				curr_path[level] = i
				visited[i] = True
				
				# call TSPRec for the next level
				TSPRec(adj, curr_bound, curr_weight, 
					level + 1, curr_path, visited)

			# Else we have to prune the node by resetting 
			# all changes to curr_weight and curr_bound
			curr_weight -= adj[curr_path[level - 1]][i]
			curr_bound = temp

			# Also reset the visited array
			visited = [False] * len(visited)
			for j in range(level):
				if curr_path[j] != -1:
					visited[curr_path[j]] = True

# This function sets up final_path
def TSP(adj):
	
	# Calculate initial lower bound for the root node 
	# using the formula 1/2 * (sum of first min + 
	# second min) for all edges. Also initialize the 
	# curr_path and visited array
	curr_bound = 0
	curr_path = [-1] * (N + 1)
	visited = [False] * N

	# Compute initial bound
	for i in range(N):
		curr_bound += (firstMin(adj, i) +
					secondMin(adj, i))

	# Rounding off the lower bound to an integer
	curr_bound = math.ceil(curr_bound / 2)

	# We start at vertex 1 so the first vertex 
	# in curr_path[] is 0
	visited[0] = True
	curr_path[0] = 0

	# Call to TSPRec for curr_weight 
	# equal to 0 and level 1
	TSPRec(adj, curr_bound, 0, 1, curr_path, visited)

# Driver code

# Adjacency matrix for the given graph
adj = graph.adjMatrix
N = graph.numVertices

# final_path[] stores the final solution 
# i.e. the // path of the salesman.
final_path = [None] * (N + 1)

# visited[] keeps track of the already
# visited nodes in a particular path
visited = [False] * N

# Stores the final minimum weight
# of shortest tour.
final_res = maxsize

TSP(adj)

print("Minimum cost :", final_res)
print("Path Taken : ", end = ' ')
for i in range(N + 1):
	print(final_path[i], end = ' ')

# This code is contributed by ng24_7


Minimum cost : 4056.6809443061284
Path Taken :  0 1 6 2 7 8 9 10 11 3 5 4 0 